In [ ]:
%load_ext autoreload
%autoreload 2
import json
import pandas as pd
from bs4 import BeautifulSoup
import local.job_scrape_funcs as scrape
import local.analysis as ana
import local.models as models
from datetime import datetime
from pathlib import Path

def job_pipeline(job_title, location, post_time, pages, save_path, resume_path, word_scores):
    #TODO: try except statements
    # Scrape jobs:
    job_data = scrape.scrape_jobs(pages=pages, job_title=job_title, location=location, post_time=post_time)
    date = datetime.today().strftime("%Y-%m-%d")

    # Save:
    index = 1
    full_path = save_path+str(job_title).replace(" ","-")+"_"+str(date)+"_"+str(index)+".json"
    while (Path.cwd()/full_path).exists():
        index += 1
        full_path = save_path+str(job_title).replace(" ","-")+"_"+str(date)+"_"+str(index)+".json"

    print("Saving to ", full_path)
    with open(full_path, 'w', encoding='utf-8') as f:
        json.dump(job_data, f, ensure_ascii=False, indent=4)

    # Find matches:
    with open(full_path, 'r', encoding="utf-8") as f:
        data = json.load(f)

    raw_df = pd.DataFrame(data["jobs"])
    df = ana.find_job_match(models.embed_model, models.nlp_model, raw_df, resume_path, models.stop_words, word_scores)
    print("Generating report:\n\n")
    html = ana.generate_html_report(df)
    soup = BeautifulSoup(html)
    with open("output1.html", "w", encoding="utf-8") as file:
        file.write(str(soup))
    return df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
word_scores = {
    "phd": 1.2,
    "python": 1.1,
    "senior": 0.8,
}

In [10]:
df = job_pipeline(job_title="machine learning engineer", 
                  location="Chicago", 
                  post_time=3, 
                  pages=40,
                  save_path="data/job_queries/", 
                  resume_path="data/resume.pdf",
                  word_scores=word_scores)

Scraping page 0
Job filtered out
9 scraped successfully.
Scraping page 1
Job filtered out
18 scraped successfully.
Scraping page 2
28 scraped successfully.
Scraping page 3
Job filtered out
Job filtered out
36 scraped successfully.
Scraping page 4
46 scraped successfully.
Scraping page 5
Job filtered out
55 scraped successfully.
Scraping page 6
Job filtered out
64 scraped successfully.
Scraping page 7
Job filtered out
Job filtered out
72 scraped successfully.
Scraping page 8
Job filtered out
81 scraped successfully.
Scraping page 9
Job filtered out
90 scraped successfully.
Scraping page 10
Job filtered out
Job filtered out
98 scraped successfully.
Scraping page 11
Job filtered out
Job filtered out
Job filtered out
Job filtered out
Job filtered out
103 scraped successfully.
Scraping page 12
Job filtered out
Job filtered out
Job filtered out
Job filtered out
109 scraped successfully.
Scraping page 13
Job filtered out
Job filtered out
Job filtered out
Job filtered out
Job filtered out
Job 

In [ ]:
# Scrape job listings:
jobs = scrape.scrape_jobs(pages=2, job_title="Data Scientist", location="Chicago", post_time=5)

In [ ]:
# Save to JSON:
with open("data/job_queries/job_data_test.json", 'w', encoding='utf-8') as f:
    json.dump(jobs, f, ensure_ascii=False, indent=4)

In [5]:
# Load JSON:
with open("data/job_queries/Data-Scientist_2025-04-23_1.json", 'r', encoding='utf-8') as f:
    jobs = json.load(f)
raw_df = pd.DataFrame(jobs["jobs"])

In [6]:
# Find job matches:
df = ana.find_job_match(models.embed_model, models.nlp_model, raw_df, "data/resume.pdf", models.stop_words, word_scores)

In [7]:
# generate job match report
html = ana.generate_html_report(df)

In [8]:
soup = BeautifulSoup(html)
with open("output1.html", "w", encoding="utf-8") as file:
    file.write(str(soup))